In [16]:
from itertools import chain
from collections import Counter
import operator

# Define All Function

In [2]:
def get_data(filename):
    """Fungsi untuk melakukan load data pada file .tsv maupun .txt
    
    File yang digunakan yaitu file .tsv maupun file yang mampu dibuka
    menggunakan teks editor. Dalam file tersebut terdapat tag pembuka <kalimat>
    dan diakhiri tag penutup </kalimat> yang berfungsi sebagai penanda bahwa
    entitas tersebut termasuk ke dalam satu kalimat.
    
    Args:
        filename: string dari nama file yang akan diload datanya.
        
    Return:
        list dari kata-kata dan tags dengan index yang menunjukkan posisi kalimat tersebut.
    """
    
    # Load data dan buka sebagai file
    sentences = []
    tags = []
    with open(filename) as file:
        contents = file.readlines()

    # Hapus karakter \n yang tidak dibutuhkan
    contents = [content.strip() for content in contents]
    idx = 0
    while idx < len(contents):
        word = []
        tag = []
        # looping sampai menemukan pattern dengan awalan </kalimat
        while not contents[idx].startswith('</kalimat'):
            # kondisi jika menemukan sebuah data yang tidak memiliki awalan <kalimat
            if not contents[idx].startswith('<kalimat'):
                temp_word, temp_tag = contents[idx].split("\t")
                word.append(temp_word.lower())
                tag.append(temp_tag)
            idx += 1
        sentences.append(word)
        tags.append(tag)
        idx += 2
        
    return sentences, tags

In [3]:
def flatten(multi_list):
    """Mengembalikan list multi dimensi ke dalam list satu dimensi.
    
    Input list yang masuk di proses menggunakan method from_iterable dari
    package itertools dan mengembalikan object berupa generator yang iterable.
    
    Args:
        multi_list: list multi dimensi.
    
    Return:
        list satu dimensi yang nantinya digunakan untuk pemrosesan lebih lanjut.
    """
    
    return chain.from_iterable(multi_list)

In [4]:
def get_emission_table(words, tags):
    """Membuat tabel emisi dari input kata dan tags.
    
    Fungsi untuk mengembalikan sebuah tabel dari kata dan kemunculan tags terhadap kata tersebut.
    
    Args:
        words: list kata.
        tags: list tags.
    
    Return:
        dictionary yang merepresntasikan tabel emisi.
    """
    
    # Membuat representasi tabel emission probability dari HMM
    hidden_state = {}
    for word in words:
        # Jika word belum pernah ditemui maka akan digenerate
        # tagset berserta probabilitasnya
        if word not in hidden_state.keys():
            word_tags = []
            for idx, wrd in enumerate(words):
                if wrd == word :
                    word_tags.append(tags[idx])
            # Membuat dictionary berisi semua tagset dari sebuah word
            # pada korpus. Berserta jumlah kemunculan setiap tagsetnya
            tag_count = {}
            for tag in word_tags:
                if tag not in tag_count.keys():
                    tag_count[tag] = 1
                else:
                    tag_count[tag] += 1
            total = 0
            # Mengubah jumlah kemunculan menjadi probabilitas
            tag_prob = {}
            for count in tag_count.values():
                total += count
            for tagset in tag_count.keys():
                tag_prob[tagset] = tag_count[tagset]/total
            hidden_state[word] = tag_prob
        else:
            # Jika word sudah pernah ditemui maka akan dilewati
            continue
    return(hidden_state)

# Load and Separate Dataset

Load dan pisahkan dataset sebanyak 1000 data train dan 20 data test

In [5]:
filename = '../data/corpus.tsv'
sentences, tags = get_data(filename)

In [6]:
# mengambil 1020 data awal
data_train, data_test, tags_train, tags_test = sentences[:1000], sentences[1000:1020], tags[:1000], tags[1000:1020]

In [7]:
data_train = list(flatten(data_train))
tags_train = list(flatten(tags_train))

Tabel emisi dibuat berdasarkan data train yang telah diambil pada corpus

In [8]:
hidden_table = get_emission_table(data_train, tags_train)

# Baseline Model

In [9]:
def predict_common_tag(data, emmision_table):
    """Fungsi untuk memprediksi suatu tag dengan pendekatan baseline.
    
    Pendekatan baseline memungkinkan prediksi suatu tag dengan menghitung kemunculan suatu tag
    dalam corpus.
    
    Args:
        data: list berisi kata yang akan diprediksi.
        emmision_table: dictionary berisi tabel emisi
    
    Return:
        prediksi tags dari list data input
    """
    
    # dengan menggunakan hidden table untuk membangun baseline model berdasarkan corpus
    predict_tags = []
    for word in data:
        if word not in emmision_table.keys():
            predict_tags.append('NN')
        else:
            most_common_tag = max(emmision_table[word].items(), key=operator.itemgetter(1))[0]
            predict_tags.append(most_common_tag)
    
    return predict_tags

In [10]:
def get_accuracy_baseline(predict_list, test_list):
    """Fungsi untuk menghitung akurasi dari model baseline
    
    Hasil prediksi dari model baseline dibandingkan dengan data test yang berisi 
    tags sesungguhnya.
    
    Args:
        predict_list: list dari prediksi menggunakan baseline
        test_list: list berisi tags yang digunakan untuk validasi model
    
    Return:
        Mengembalikan nilai akurasi dari model yang dibangun
    
    """
    count = 0
    for i in range(len(predict_list)):
        if (predict_list[i] == test_list[i]):
            count += 1
    
    accuracy = count / len(test_list)
    return accuracy

Hitung akurasi berdasarkan data test

In [11]:
predict = []
for sentence in data_test:
    predict_tags = predict_common_tag(sentence, hidden_table)
    predict.append(predict_tags)

In [12]:
flatten_tags_test = list(flatten(tags_test))
flatten_predict = list(flatten(predict))

In [13]:
akurasi = get_accuracy_baseline(flatten_predict, flatten_tags_test)
print("Akurasi model baseline :",akurasi)

Akurasi model baseline : 0.8544117647058823


Output hasil prediksi

In [15]:
print("Kalimat:", ' '.join(data_test[6]))
print("Prediksi:", ' '.join(predict[6]))

Kalimat: fitch ratings , dalam siaran pers -nya , senin , mengatakan bahwa prospek rating kedua bank itu tetap di stabil .
Prediksi: NNP NNP Z IN NN PRP Z NNP Z VB SC NN FW CD NNP PR RB IN JJ Z
